# Pursuit Evasion Game Evolutionary Algorithm

### Pursuit Evasion Games

Pursuit Evasion Games are a subclass of non-cooperative games. The simplest case includes a pursuer and an evader, which in our case are referred to as the predator and prey respectively. The pursuer's objective is to catch the evader, while the evader must avoid capture. This is a zero-sum game of competing interests. The characteristics of the two players are different. In our experiment we give the predator an advantage in top speed, whereas the prey has an agility advantage, encoded as having a higher acceleration. 

### Game Structure
The game has been defined in the `00_Deterministic` notebook, where we also added functionality to simulate, plot, and perform signal processing through discrete cosine transform. The purpose of `00_Deterministic` is to explore the different behaviors that can emerge from this Differential Game. In the next section we will take this game, tweak its architecture and propose an evolutionary algorithm to find the predator and prey's best parameters.

In [1]:
# imports
import numpy as np

## Genomes

The first step to evolution is a basic structure to hold the genetic information of the individuals. For this reason, a unqiue genome needs to be defined for the prey and predator. 

**Predator** \
The predator has the max speed and acceleration parameters, to find the best speed and acceleration combination, as well as a `predicition_factor`, which could be described as how much to look ahead at where the prey could be, and finally a `pursuit_strategy`, what the best way to follow the prey is.

**Prey** \
The prey also has the max speed and acceleration parameters, but is also equipped with the ability to change its `reaction_radius`, how close the predator has to be for it to start evading, and its `evasion_angle`, the direction the prey will take to evade.

In [2]:
class PredatorGenome:
    def __init__(self):
        self.max_speed = 0.0
        self.max_acceleration = 0.0

        self.prediction_factor = 0.0
        self.pursuit_strategy = 0.0

class PreyGenome:
    def __init__(self):
        self.max_speed = 0.0
        self.max_acceleration = 0.0

        self.react_radius = 0.0
        self.evasion_angle = 0.0

        # experiment with pursuit strategy ?


## Fitness Function

The next important part of an evolutionary algorithm is the fitness function, to quantify how well the current genetic combination of prey and predator are performing. This metric will be important in deciding the "survival of the fittest", the best genomes that will go on to reproduce and create the new offspring of agents. Again, we will need a unique way of assessing the fitness of predators and prey, as they have distinct, and competing, interests.

**Predator** \
The predator's goal is to catch the prey, so we can start by giving the predator `0` reward if the prey is not caught within the allotted time or distance. From this, a natural fitness measure could be the `capture_time`, where the lower this value is, the better the predator's performance. In addition, we could also look at the predator's `pursuit_efficiency`, which is defined by dividing the straight line distance between the predator's initial position and the eventual capture, and the distance that the predator actual travelled within the simulation.

**Prey** \  
For the prey there are a few metrics that could help us understand how well it is evading its pursuer. For one, `survival_time` is crucial, because the longer the prey evades capture, the more effectively it is out of reach of the predator's hungry fangs. We could also look at `total_distance` travelled, which would tell us how far the prey got before it was taken down. Another metric could be a calculation of the `average_distance` the prey was able to keep between itself and the predator. One can imagine that a few strategies may arise for the prey, namely, getting as far away from the predator, and maintaining that distance, but also being just out of reach for the predator, making use of its agility. Because of this, we will be implementing a more nuanced fitness function, rewarding both distance and time away from the predator.

In [3]:
# Predator Fitness
def predator_fitness(simulation_result, w_time=0.5, w_eff=0.5):
    if not simulation_result.prey_captured:
        return 0.0 # no capture, no reward
    
    capture_time = 1.0 / (simulation_result.steps_until_capture * simulation_result.params['dt'])

    # straight-line distance from start to prey capture
    start_pos = simulation_result.state[0, 0, 0, :]
    capture_pos = simulation_result.state[simulation_result.steps_until_capture, 0, 0, :]
    optimal_distance = np.linalg.norm(capture_pos - start_pos)
    
    # actual distance traveled
    positions = simulation_result.state[:simulation_result.steps_until_capture, 0, 0, :]
    actual_distance = np.sum(np.linalg.norm(positions[1:] - positions[:-1], axis=1))
    pursuit_efficiency = optimal_distance / actual_distance

    return (capture_time * w_time) + (pursuit_efficiency * w_eff)

# Prey Fitness
def prey_fitness(simulation_result, params, w_surv=0.6, w_avgd=0.2, w_totd=0.2):
    survival_time = simulation_result.steps_until_capture * params['dt']
    avg_distance = np.mean(positive_distance(simulation_result.state))
    prey_positions = simulation_result.state[:, 1, 0, :]
    total_distance = np.sum(np.linalg.norm(prey_positions[1:] - prey_positions[:-1], axis=1))
    
    return (survival_time * w_surv) + (avg_distance * w_avgd) + (total_distance * w_totd)

## Evolutionary Operators

Next we need to define the evolutionary operators: selection, crossover, and mutation.